## Using the DBSCAN algorithm to cluster localizations.

This notebook demonstrates how to cluster localizations using the DBSCAN algorithm. It also demonstrates how to work with the clustered data.

Note:
* The algorithm works in 2D or 3D, but in this example we just do 2D clustering.

References:
* [DBSCAN (Wikipedia)](https://en.wikipedia.org/wiki/DBSCAN).
* [Ester et al, Proceedings of the Second International Conference on Knowledge Discovery and Data Mining, 1996](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.121.9220).

### Configuration

Create an empty directory somewhere on your computer and tell Python to go to that directory.

In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot
import numpy
import os

os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

numpy.random.seed(1)


### Generate data to cluster

In this example we are just going to generate the clustering data synthetically.

In [ ]:
import storm_analysis.jupyter_examples.clustering_data as clusteringData

clusteringData.makeClusters("clusters.hdf5", 40, 1000, 20000)

In [ ]:
# Make an image from the data.
import storm_analysis.sa_utilities.hdf5_to_image as h5_image

sr_im = h5_image.render2DImage("clusters.hdf5", scale = 2, sigma = 1)

fig = pyplot.figure(figsize = (9, 6))
pyplot.imshow(sr_im, cmap = "gray")
pyplot.show()

### Cluster the data

Note:
* The results of the clustering are saved in the HDF5 that contained the tracks / localizations.
* Clustering is done on tracks if they are available, otherwise it is done on the localizations.

In [ ]:
import storm_analysis.dbscan.dbscan_analysis as dbscanAnalysis

# The second parameter is the DBSCAN eps value in nanometers.
# The third parameter is the DBSCAN mc value.
dbscanAnalysis.findClusters("clusters.hdf5", 100.0, 50)

### RGB image of the clustering results

In [ ]:
import storm_analysis.dbscan.cluster_images as clusterImages

[rgb_im, sum_im, num_clusters] = clusterImages.clusterImages("clusters.hdf5", 10, 3, scale = 2, 
                                                             show_unclustered = True)


fig = pyplot.figure(figsize = (9, 6))
pyplot.imshow(rgb_im, cmap = "gray")
pyplot.show()


### Create a file with some statistics for each cluster

In [ ]:
stats_name = dbscanAnalysis.clusterStats("clusters.hdf5", 10)

print()
print("Cluster statistics:")
with open(stats_name) as fp:
    for line in fp:
        print(line.strip())

### Working with HDF5 clusters files

In [ ]:
import storm_analysis.dbscan.clusters_sa_h5py as clSAH5Py

# The SAH5Clusters object is a sub-class of the SAH5Py object, so it provides all
# the methods of SAH5Py object in addition to a few of it's own.
#
with clSAH5Py.SAH5Clusters("clusters.hdf5") as cl_h5:
    
    # Get clustering program information.
    print("Analysis info", cl_h5.getClusteringInfo())
    
    # Get the number of clusters.
    print("Total clusters", cl_h5.getNClusters())
    
    # This is the recommended way to iterate over all the clusters. Like the tracks
    # and localizations iterators you can specify which fields you want if you don't
    # want to get them all.
    #
    print()
    for index, cluster in cl_h5.clustersIterator(min_size = 100):
        print("cluster {0:0d} <x> = {1:.3f}".format(index, numpy.mean(cluster['x'])))
        if (index >= 5):
            break
            
    # Note that if you only need the fields that are stored with each cluster this
    # iteration can be much faster. This is because the other fields have to be 
    # looked up from each localization/track that is in the cluster.
    
    # These are the fields that are stored with each cluster.
    print()
    print("Cluster fields:", cl_h5.getClustersFields())
    
    # This should be noticeably faster. As 'x' is available in the cluster so we
    # don't have to go the through the tracks to get it.
    print()
    for index, cluster in cl_h5.clustersIterator(min_size = 100, fields = ['x']):
        print("cluster {0:0d} <x> = {1:.3f}".format(index, numpy.mean(cluster['x'])))
        if (index >= 5):
            break